# Intermovie - projet 6 
Utilisation de dataframes pour manipuler des données. Exportation en CSV pour chaque dataframe créé. 

## Import des librairies

In [ ]:
import pandas as pd

%load_ext autoreload
%autoreload 2

## 2. Les notes moyennes des différents genres
### Import des datas initiaux :

In [ ]:
title_basics = pd.read_csv("../movies_dataset/title.basics.tsv",sep='\t', index_col='tconst', usecols=['tconst', 'genres', 'titleType'])
print('Title Basics imported.')

title_ratings = pd.read_csv("../movies_dataset/title.ratings.tsv",sep='\t', index_col='tconst', usecols=['tconst', 'averageRating'])
print('Title Ratings imported.')

### Nettoyage du dataset :

In [ ]:
title_basics = title_basics.dropna()
title_basics = title_basics[['genres']][title_basics.titleType.str.contains('movie|tvMovie', regex=True)]

### Jointure des tables :

In [ ]:
temporary = title_basics[['genres']].merge(title_ratings, how='left', left_index=True, right_index=True)
temporary = temporary.dropna()
temporary = temporary[temporary.genres != '\N']

In [ ]:
mean_genre = pd.DataFrame(temporary.genres.str.split(',').tolist(), index=temporary.index).stack()
mean_genre = mean_genre.reset_index(['tconst', 0])

temporary = temporary.drop(columns=['genres'])

mean_genre = mean_genre.set_index('tconst')
mean_genre.columns = ['genre']

mean_genre = mean_genre.merge(temporary, how='left', left_index=True, right_index=True)
mean_genre = mean_genre[mean_genre.genre != '\N']

In [ ]:
mean_genre = mean_genre.groupby('genre').mean()

### Import en CSV du fichier

In [ ]:
mean_genre.sort_values(by=['averageRating'], ascending=False)

In [ ]:
mean_genre.to_csv('mean_genre.csv')